In [1]:
embed_dim=100
G=4
#batch_size=80
number_of_negative_tokens = 5
n_epochs = 30
n_workers=64
queue_size = 64
padding_length=3
title_length_percentile=95
desc_length_percentile=90

import os, time, json,pandas as pd, numpy as np, re, string, random,sys
from collections import Counter,defaultdict
import Levenshtein


from keras.layers import Input,Lambda,Conv1D,GlobalMaxPool1D,concatenate,Dense,Dropout,PReLU,TimeDistributed,Embedding,Activation
from keras import Model
import keras.backend as K
import keras
from keras.utils import Sequence
import tensorflow as tf



df= pd.read_csv('allrecipes.csv')


# In[7]:


/Users/jermainemarshall/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [84]:
df.WHO.value_counts()

1    12752
2    10950
3     6194
4     3017
0     1600
5     1412
6      444
7       60
Name: WHO, dtype: int64

In [85]:
pd.options.display.max_colwidth= 500
df.head(1)

,RECIPE,WHO
0,"mayonnaise#237.0,parmigiano reggiano cheese#75.0,granulated sugar#25.200000000000003,romaine lettuce#283.0,bacon bits#56.0",1


In [86]:
#!pip3 install python-levenshtein
#!pip3 install setuptools


# strip punctuation and combine ingredients into one word separated by a spae

#df['RECIPE'] = df['RECIPE'].str.replace('\d+', '')

#df['RECIPE'] = df['RECIPE'].str.replace(' ', '')
df['RECIPE'] = df['RECIPE'].str.replace(' ', '')
df['RECIPE'] = df['RECIPE'].str.replace('#', ' ')
#df['RECIPE'] = df['RECIPE'].str.replace(r'[\s^\w]', '')
#df['RECIPE'] = df['RECIPE'].str.replace(' ', '')
df['RECIPE'] = df['RECIPE'].str.replace(',', ' ')
#df['RECIPE'] = df['RECIPE'].str.replace('\d', ' ')

In [87]:
df.head(12)

,RECIPE,WHO
0,mayonnaise 237.0 parmigianoreggianocheese 75.0 granulatedsugar 25.200000000000003 romainelettuce 283.0 baconbits 56.0,1
1,PhiladelphiaCreamCheese 454.0 fineseasalt 3.0 pecanhalves 164.0,1
2,hardboiledeggs 100.0 wholemilk 231.31199999999998 cakeflour 83.0 fineseasalt 0.4 vegetableoil 6.8,3
3,zucchini 646.0 extra-virginoliveoil 13.5 hotwater 58.21333333333333 fineseasalt 0.4,3
4,chiliflakes 1.7999999999999998 fineseasalt 0.4 honey 339.0 saltedbutter 114.0 srirachasauce 153.4,0
5,granulatedsugar 403.0 cinnamon 2.6 gratednutmeg 2.2 fineseasalt 0.4 saltedbutter 114.0 purevanillaextract 12.600000000000001,3
6,finelychoppedonion 150.0 fineseasalt 12.0 chiliflakes 1.7999999999999998 largegarliccloves 2.72,2
7,orangejuice 117.056 limewedges 33.5 honey 20.34 chiliflakes 1.7999999999999998 chickenbreasts 1536.0 coriander 0.24,3
8,honey 61.019999999999996 lowsodiumsoysauce 45.0 balsamicvinegar 48.0 largegarliccloves 8.16 freshlygroundpepper 2.3,2
9,garlicpowder 9.3 chilipowder 8.100000000000001 vegetableoil 13.6,4


In [88]:
df['WHO'] = df['WHO'].replace({0:'zero', 1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', 6: 'six', 7: 'seven'})

In [89]:
pd.options.display.max_colwidth= 500
df.head(4)

,RECIPE,WHO
0,mayonnaise 237.0 parmigianoreggianocheese 75.0 granulatedsugar 25.200000000000003 romainelettuce 283.0 baconbits 56.0,one
1,PhiladelphiaCreamCheese 454.0 fineseasalt 3.0 pecanhalves 164.0,one
2,hardboiledeggs 100.0 wholemilk 231.31199999999998 cakeflour 83.0 fineseasalt 0.4 vegetableoil 6.8,three
3,zucchini 646.0 extra-virginoliveoil 13.5 hotwater 58.21333333333333 fineseasalt 0.4,three


In [90]:
from sklearn.model_selection import train_test_split
#(X_train, X_test) = train_test_split(df2,test_size=0.20)


#with tf.device('/gpu:0'):
X_train, X_validation, X_test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

descTokenCounts = pd.Series(' '.join(df['RECIPE']).split())

descTokenCounts= descTokenCounts.value_counts().to_dict()

df= df.drop_duplicates()


df= df.dropna()


def remove_non_ascii_1(text):

    return ''.join([i if ord(i) < 128 else ' ' for i in text])

def remove_non_ascii_2(text):

    return re.sub(r'[^\x20-\x7E]+',' ', text)

vocabSize=100000
tokens = sorted(descTokenCounts.items(),key=lambda x:x[1],reverse=True)[:vocabSize]

tokens = [i[0] for i in tokens]
tokens.append('token_unknown')
token_to_index = {}
index_to_token = {}


df['WHO'] = ([remove_non_ascii_1(sentence) for sentence in df['WHO']])
df['WHO'] = ([remove_non_ascii_2(sentence) for sentence in df['WHO']])


df['RECIPE']= ([remove_non_ascii_1(sentence) for sentence in df['RECIPE']])


# In[13]:




df['RECIPE']= ([remove_non_ascii_2(sentence) for sentence in df['RECIPE']])



In [91]:
model_name = 'allmy_jermaine_model1_jerm'


from sklearn.model_selection import train_test_split
#(X_train, X_test) = train_test_split(df2,test_size=0.20)


#with tf.device('/gpu:0'):
X_train, X_validation, X_test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])



with open('alltrain_data.json', 'wb') as f:
    f.write(X_train.to_json(orient='records', lines=True).encode('ascii', 'ignore'))

    
with open('alltest_data.json', 'wb') as g:
    g.write(X_test.to_json(orient='records', lines=True).encode('ascii', 'ignore'))


    
with open('allvalidation_data.json', 'wb') as t:
    t.write(X_validation.to_json(orient='records', lines=True).encode('ascii', 'ignore'))

def loadData(dataPath):
    data=[]
    start_time = time.time()
    with open(dataPath) as f:
        for line in f:
            data.append(line)
    loadTime = time.time() - start_time

    #print loadTime
    return data

training_data = loadData('alltrain_data.json')
validation_data = loadData('allvalidation_data.json') 
test_data = loadData('alltest_data.json') 


In [92]:
title_length= df['WHO'].map(len)
desc_length= df['RECIPE'].map(len)
titleCounts = df['WHO'].value_counts().to_dict()

charset=set(list(string.ascii_lowercase)+list(map(str,range(10)))+list("?$*!: @ ^ %- / <> ~ ` \\ [] {} \n = | \" _+#&-/\n %() ' '.;,"))
title_char_to_index_dict={c:i for i,c in enumerate(charset)}
title_index_to_char_dict={i:c for c,i in title_char_to_index_dict.items()}

In [93]:
print(title_char_to_index_dict)

{'z': 0, 'x': 1, 'o': 2, ',': 3, 's': 4, 'w': 5, '5': 6, '6': 7, '<': 8, ')': 9, '.': 10, 'u': 11, '}': 12, '=': 13, 'c': 14, '>': 15, 't': 16, '\\': 17, 'd': 18, '3': 19, 'p': 20, '7': 21, 'm': 22, '9': 23, 'q': 24, '/': 25, '#': 26, '&': 27, '_': 28, '(': 29, ' ': 30, ';': 31, 'e': 32, '^': 33, '|': 34, '~': 35, '!': 36, '+': 37, 'k': 38, '[': 39, 'f': 40, 'i': 41, 'l': 42, '8': 43, '?': 44, '%': 45, '2': 46, '*': 47, 'r': 48, 'y': 49, 'n': 50, ':': 51, 'v': 52, '-': 53, '{': 54, '1': 55, 'h': 56, '`': 57, 'a': 58, 'j': 59, '0': 60, ']': 61, '"': 62, "'": 63, 'b': 64, 'g': 65, '4': 66, '\n': 67, '$': 68, '@': 69}


In [94]:



for i in range(len(tokens)):
    token_to_index[tokens[i]] = i+1
    index_to_token[i+1] = tokens[i] 


with open('allmodels_indices.json'.format(model_name),'w') as f:
    json.dump({'title_char_to_index_dict':title_char_to_index_dict,'title_index_to_char_dict':title_index_to_char_dict,              'token_to_index':token_to_index,'index_to_token':index_to_token},f)


    
title_max_len = np.ceil(np.percentile(title_length, title_length_percentile)).astype(int)

def title_char_tokenizer(entity,entity_max_len):
    preproced_entity=(" "*padding_length+entity)[:entity_max_len]
    padded_entity=preproced_entity+"".join([" "]*(entity_max_len-len(preproced_entity)))
    return [title_char_to_index_dict[i] for i in padded_entity]


## RECIPE tokenizer
## 0 padded in the end
desc_max_length = np.ceil(np.percentile(desc_length, desc_length_percentile)).astype(int)
def job_description_tokenizer(desc,entity_max_len):
    desc = re.sub("[,\/]",' ',desc).split()
    desc = [token_to_index[i] if i in token_to_index else token_to_index['token_unknown'] for i in desc]
    desc=(desc+[0]*entity_max_len)[:entity_max_len]
    return desc

In [95]:
with open ('allallmodels_model_params.json'.format(model_name),'w') as f:
    json.dump({'title_max_len':int(title_max_len),'desc_max_length':int(desc_max_length)},f)
    
## character embedding model for a list of titles
def char_embed_model_seq(input_layer,char_length,char_index_dict,entity_name,n_filters=512,embed_dim=embed_dim):
    '''
    Same as char_embed_model, but produces embeddings for a sequence of say, titles.
    Applies the same layers (as char_embed_model) to each member of the sequence
    '''    
    entity_onehot_layer=TimeDistributed(Lambda(lambda x: K.one_hot(x,len(char_index_dict)),output_shape=(char_length,len(char_index_dict))),name=entity_name+'_onehot_seq')(input_layer)
        
    conv7_layer=TimeDistributed(Conv1D(filters=n_filters,kernel_size=4,activation='relu'),name=entity_name+'_conv_7seq')(entity_onehot_layer)
    maxpool7_layer=TimeDistributed(GlobalMaxPool1D(),name=entity_name+'maxpool1seq')(conv7_layer)

    conv3_layer=TimeDistributed(Conv1D(filters=n_filters,kernel_size=2,activation='relu'),name=entity_name+'_conv_3seq')(entity_onehot_layer)
    maxpool3_layer=TimeDistributed(GlobalMaxPool1D(),name=entity_name+'maxpool2seq')(conv3_layer)

    pools_concat=keras.layers.concatenate([maxpool7_layer,maxpool3_layer],name=entity_name+'_maxpools_concatseq')
    #pools_dropped=(pools_concat)

    entity_dense_layer=TimeDistributed(Dense(n_filters*2,activation='relu'),name=entity_name+'_intermediate_denseseq')(pools_concat)
    
    dropout_layer=TimeDistributed(Dropout(0.4),name=entity_name+"_droppedseq")(entity_dense_layer)
    
    entity_final_dense=TimeDistributed(Dense(embed_dim),name=entity_name+'_last_denseseq')(dropout_layer)
    entity_prelu_layer=TimeDistributed(PReLU(),name=entity_name+'_preluseq')(entity_final_dense)
    embed_norm_layer=TimeDistributed(Lambda(lambda x: x/(K.epsilon()+K.sqrt(K.batch_dot(x,x,axes=-1)))),name=entity_name+'_unit_norm_embedseq')(entity_prelu_layer)

    entity_embed_model=keras.models.Model(input_layer,embed_norm_layer,name=entity_name+"_embedding_model")
    return entity_embed_model




# In[25]:


## job describtion embedding - average word embedding (remove 0 padding)
def desc_embed_model(input_layer,vocab_size,entity_name,embed_dim=embed_dim):
    wordEmbed = Embedding(input_dim = vocab_size,output_dim = embed_dim, name='word_embedding')(input_layer) 
    
    ## get average embedding for desc (remove 0 padding)
    mask=Lambda(lambda x: K.cast(K.minimum(1,K.cast(x,'int32')),'float32'),name='mask')(input_layer)
    mask_summed=Lambda(lambda x: 1/(K.epsilon()+K.sum(x,axis=-1)),name='mask_summed')(mask)
    mask_normed=keras.layers.multiply([mask,mask_summed],name='mask_normed')
    desc_embedding_averaged=keras.layers.dot([wordEmbed,mask_normed],axes=1,name='desc_embedding_averaged')
    
    desc_embed_model=Model(input_layer,desc_embedding_averaged,name=entity_name+"_embedding_model")
    return desc_embed_model
# In[26]:


alpha=0.5
sampling_titles = titleCounts.keys()

sampling_titles= [k for k in titleCounts.keys()]


# In[27]:


sampling_titles_array = np.array(sampling_titles)


# In[28]:


sampling_prob_array = np.array(list(titleCounts.values()))
sampling_prob_array = np.power(sampling_prob_array,alpha)
sampling_prob_array = sampling_prob_array/sum(sampling_prob_array)


# In[29]:


def sampling_based_on_similarity(title,sub_sampling_titles,number_of_negative_tokens=5):
## Randomly sample sub_sampling_titles based on emperical distribution
## Randomly sample titles from the selected sub-titles based on Levenshtein similarity
    st = time.time()
    sampling_titles = np.array(sub_sampling_titles)

    sampling_prob = np.array([Levenshtein.ratio(title,i) for i in sub_sampling_titles])#Levenshtein.ratio(title,i) for i in sub_sampling_titles])
    sampling_prob = sampling_prob/sum(sampling_prob)
    
    samples=np.random.choice(sampling_titles,size=number_of_negative_tokens,p=sampling_prob)
  
    return samples.tolist()



In [96]:
class empirical_sampling_gen(Sequence):

    def __init__(self, data, batch_size,number_of_negative_tokens,sampling_titles,sampling_prob,sub_sampling_size):
        '''        
        batch_size: number of resumes to process per batch        
        The actual number of training samples varies as each resume is splitted into sub-sequences
        '''
        self.batch_size = batch_size
        self.input_data = data
        self.data_length = len(data)
        self.number_negative_tokens = number_of_negative_tokens
        self.sampling_titles = sampling_titles
        self.sampling_prob = sampling_prob
        self.sub_sampling_size = sub_sampling_size
    def __len__(self):
        return int(np.ceil(self.data_length / float(self.batch_size)))
    

    def __getitem__(self, idx):
        i = idx
        batch_size = self.batch_size 
        desc_input_batch=[]
        title_input_batch=[]
        y_pos_batch=[]
        y_neg_batch=[] 
        titles = []
        for k in range(i * batch_size, min((i + 1) * batch_size, self.data_length)):
            line = json.loads(self.input_data[k])
            desc = job_description_tokenizer(line['RECIPE'],desc_max_length)
            #desc = str(desc)
            titles.append(line['WHO'])
            title = title_char_tokenizer(line['WHO'],title_max_len)
            #title= str(title)
            desc_input_batch.append(desc)
            title_input_batch.append([title]*self.number_negative_tokens)
            y_pos_batch.append(1)
            y_neg_batch.append([0]*self.number_negative_tokens)  
            
        ## Sample negative titles for each batch all together              
        neg_titles = np.random.choice(self.sampling_titles,                                                 size=len(titles)*self.number_negative_tokens,                                                 p=self.sampling_prob)
        
        ## Resample for titles with length below median title length based on Levenshtein similarity
        sub_sampling_titles = np.random.choice(self.sampling_titles,p=self.sampling_prob,                                               size=self.sub_sampling_size*len(titles),replace=True)
        for i,t in enumerate(titles):
            if np.random.random()>0.5:
                sub_titles = sub_sampling_titles[i*self.sub_sampling_size:(i+1)*self.sub_sampling_size]
                samples = sampling_based_on_similarity(t,sub_titles)
                neg_titles[i*self.number_negative_tokens:(i+1)*self.number_negative_tokens]=samples
        
        neg_title_input_batch = [title_char_tokenizer(t,title_max_len) for t in neg_titles]
        neg_title_input_batch = [neg_title_input_batch[x:x+self.number_negative_tokens]                                  for x in range(0, len(neg_title_input_batch), self.number_negative_tokens)]
        
        
             
        ## Convert inputs and output to numpy arrays
        inputs = list(map(np.array,[desc_input_batch,title_input_batch,neg_title_input_batch]))
        outputs = list(map(np.array,[y_pos_batch,y_neg_batch]))     
        #return (inputs, outputs,titles,neg_titles) 
        #print(inputs)
        return (inputs, outputs) 
    

    def on_epoch_end(self):
        #'Shuffle data after each epoch'
        np.random.shuffle(self.input_data)



# In[ ]:


# In[31]:


batch_size=80


# In[32]:


trainGen = empirical_sampling_gen(training_data,batch_size,number_of_negative_tokens,                                  sampling_titles_array,sampling_prob_array,sub_sampling_size=2200)                                 



#len(title_char_to_index_dict)


# In[33]:



testGen = empirical_sampling_gen(test_data,batch_size,number_of_negative_tokens,                                    sampling_titles_array,sampling_prob_array,sub_sampling_size=1500)


# In[34]:


validateGen = empirical_sampling_gen(validation_data,batch_size,number_of_negative_tokens,                                    sampling_titles_array,sampling_prob_array,sub_sampling_size=2200)                                 


# In[35]:


target_title_input=Input(shape=(number_of_negative_tokens,title_max_len), dtype='int32', name='targe_title_input')
neg_title_input = Input(shape=(number_of_negative_tokens,title_max_len), dtype='int32', name='ne_title_input')
title_embedding_model = char_embed_model_seq(target_title_input,title_max_len,title_char_to_index_dict,"titlee")
target_title_embed = title_embedding_model(target_title_input)
target_title_embed = Lambda(lambda x:x[:,0,:],name='title_embedding_take_first_elemente')(target_title_embed)
neg_title_embed = title_embedding_model(neg_title_input)



# In[36]:


## Get resume job description embedding
desc_input = Input(shape=(desc_max_length,),name='des_input')
desc_embedding_model = desc_embed_model(desc_input,vocabSize+2,'des')
desc_embed = desc_embedding_model(desc_input)

## Calcuate dot product and loss
desc_target_title_dot = keras.layers.dot([target_title_embed,desc_embed],axes=-1,name='des_target_dot')
pos_title_logit = Activation('sigmoid',name='po_title_logit')(desc_target_title_dot)

desc_neg_title_dot = keras.layers.dot([neg_title_embed,desc_embed],axes=-1,name='des_neg_dot')
neg_title_logit = Activation('sigmoid',name='ne_title_logit')(desc_neg_title_dot)


In [97]:
embeddings = desc_embedding_model.get_weights()[0]

In [ ]:
def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [ ]:
tsne_pl

In [98]:
np.set_printoptions(suppress=True)

In [99]:
words_embeddings = {w:embeddings[idx].flatten() for w, idx in descTokenCounts.items()}

In [18]:
   #count += 1

In [136]:
def recipe_emb(temp, words_embeddings):
    np.set_printoptions(suppress=True,
    formatter={'float_kind':'{:0.60f}'.format})
    count = 0
    v=1
    for w in temp.split():
        if w in words_embeddings:
            count *=1
            v-= ((words_embeddings[w]))
        #v= np.array(v)
    return v

In [137]:
#pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [138]:
df3= df[0:90000]

In [139]:
df3['embed_summed']=df3['RECIPE'].apply(lambda x: recipe_emb(x,words_embeddings))

/Users/jermainemarshall/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [140]:
df3.head(3)

,RECIPE,WHO,embed_summed
0,mayonnaise 237.0 parmigianoreggianocheese 75.0 granulatedsugar 25.200000000000003 romainelettuce 283.0 baconbits 56.0,one,"[1.2556392, 1.013125, 0.9881612, 1.0326676, 1.1220996, 0.92741096, 1.0696841, 0.95893633, 0.9399829, 0.85452765, 1.0248402, 0.8161403, 1.0942822, 0.9560012, 0.9015771, 0.8706607, 0.9667885, 1.1423523, 0.91212034, 1.0267096, 1.1208909, 0.96120334, 1.0034764, 0.95535743, 1.1576126, 0.9084617, 0.94192725, 0.9744645, 0.95841026, 1.0107601, 0.8751195, 0.98867327, 1.0371549, 1.0868069, 0.9068235, 1.0925081, 1.0925367, 0.80589217, 0.8248976, 0.88848144, 0.95271, 0.93624926, 0.93336254, 0.93286663, ..."
1,PhiladelphiaCreamCheese 454.0 fineseasalt 3.0 pecanhalves 164.0,one,"[0.9448704, 0.9506966, 0.9196438, 1.0018139, 0.989734, 1.0773984, 0.85884434, 1.0148004, 0.99326205, 0.9288898, 0.9527746, 0.9634197, 0.99564266, 1.148674, 1.0345888, 0.88012284, 1.0006186, 1.0065337, 1.0051363, 1.1366804, 0.9735658, 0.88369405, 0.97622985, 1.1346495, 1.0143901, 0.93725085, 0.937709, 0.9401871, 0.9497963, 1.0668398, 1.0237998, 1.0752032, 0.9698898, 1.0987651, 0.99184936, 0.9644332, 1.0457834, 0.8783677, 0.9586544, 0.936395, 1.1357902, 1.0035807, 0.93785346, 1.0506476, 0.9406..."
2,hardboiledeggs 100.0 wholemilk 231.31199999999998 cakeflour 83.0 fineseasalt 0.4 vegetableoil 6.8,three,"[1.1187569, 0.85408264, 1.1173044, 0.9470272, 1.0344192, 0.96887225, 1.0237195, 0.9795352, 0.9516653, 0.94139755, 1.0744181, 0.926491, 0.88205814, 0.8416496, 0.8900257, 1.1275212, 1.0840355, 1.1815677, 1.0186038, 1.0668372, 1.043772, 1.0040866, 1.0084603, 1.0069008, 1.15898, 1.1154802, 1.0714413, 0.8143189, 0.97373164, 0.82489383, 0.97472304, 0.87322104, 1.1389589, 0.9378641, 1.0277966, 1.1303847, 0.9410959, 0.94735867, 1.0459634, 0.9439875, 1.0121481, 0.98459184, 1.0575035, 0.8594607, 0.868..."


In [141]:
df3.embed_summed= df3['embed_summed'].astype(str)

/Users/jermainemarshall/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [142]:
df3.embed_summed.str.replace('[','')
df3['embed_summed'] = df3['embed_summed'].str.replace('[', '')
df3['embed_summed'] = df3['embed_summed'].str.replace(']', '')
df3['embed_summed'] = df3['embed_summed'].str.replace('[\n\s]+', ',')
df3['embed_summed'] = df3['embed_summed'].str.replace(r'^,', '')

/Users/jermainemarshall/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/jermainemarshall/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/jermainemarshall/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [143]:
df4 = df3['embed_summed'].apply(lambda x: pd.Series(x.split(',')))

In [144]:
df3= df3[['RECIPE','WHO']].copy()

In [145]:
df3= df3.join(df4)

In [146]:
df3.head(4)

,RECIPE,WHO,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,mayonnaise 237.0 parmigianoreggianocheese 75.0 granulatedsugar 25.200000000000003 romainelettuce 283.0 baconbits 56.0,one,1.255639195442199707031250000000000000000000000000000000000000,1.013124942779541015625000000000000000000000000000000000000000,0.988161206245422363281250000000000000000000000000000000000000,1.032667636871337890625000000000000000000000000000000000000000,1.122099637985229492187500000000000000000000000000000000000000,0.927410960197448730468750000000000000000000000000000000000000,1.069684147834777832031250000000000000000000000000000000000000,0.958936333656311035156250000000000000000000000000000000000000,...,1.081253647804260253906250000000000000000000000000000000000000,0.841649353504180908203125000000000000000000000000000000000000,0.947954654693603515625000000000000000000000000000000000000000,0.986992895603179931640625000000000000000000000000000000000000,0.903862535953521728515625000000000000000000000000000000000000,0.959453523159027099609375000000000000000000000000000000000000,0.970603942871093750000000000000000000000000000000000000000000,1.094742178916931152343750000000000000000000000000000000000000,1.052498459815979003906250000000000000000000000000000000000000,0.956190824508666992187500000000000000000000000000000000000000
1,PhiladelphiaCreamCheese 454.0 fineseasalt 3.0 pecanhalves 164.0,one,0.944870412349700927734375000000000000000000000000000000000000,0.950696587562561035156250000000000000000000000000000000000000,0.919643819332122802734375000000000000000000000000000000000000,1.001813888549804687500000000000000000000000000000000000000000,0.989733994007110595703125000000000000000000000000000000000000,1.077398419380187988281250000000000000000000000000000000000000,0.858844339847564697265625000000000000000000000000000000000000,1.014800429344177246093750000000000000000000000000000000000000,...,0.940294086933135986328125000000000000000000000000000000000000,1.006771922111511230468750000000000000000000000000000000000000,1.005477547645568847656250000000000000000000000000000000000000,1.095989704132080078125000000000000000000000000000000000000000,1.031980633735656738281250000000000000000000000000000000000000,1.040227890014648437500000000000000000000000000000000000000000,0.993172228336334228515625000000000000000000000000000000000000,1.010775566101074218750000000000000000000000000000000000000000,1.028016328811645507812500000000000000000000000000000000000000,1.062635064125061035156250000000000000000000000000000000000000
2,hardboiledeggs 100.0 wholemilk 231.31199999999998 cakeflour 83.0 fineseasalt 0.4 vegetableoil 6.8,three,1.118756890296936035156250000000000000000000000000000000000000,0.854082643985748291015625000000000000000000000000000000000000,1.117304444313049316406250000000000000000000000000000000000000,0.947027206420898437500000000000000000000000000000000000000000,1.034419178962707519531250000000000000000000000000000000000000,0.968872249126434326171875000000000000000000000000000000000000,1.023719549179077148437500000000000000000000000000000000000000,0.979535222053527832031250000000000000000000000000000000000000,...,1.130256772041320800781250000000000000000000000000000000000000,1.014510273933410644531250000000000000000000000000000000000000,1.174296975135803222656250000000000000000000000000000000000000,1.069943666458129882812500000000000000000000000000000000000000,1.010989069938659667968750000000000000000000000000000000000000,1.024985551834106445312500000000000000000000000000000000000000,1.066484451293945312500000000000000000000000000000000000000000,1.040072321891784667968750000000000000000000000000000000000000,0.947046458721160888671875000000000000000000000000000000000000,0.953204691410064697265625000000000000000000000000000000000000
3,zucchini 646.0 extra-virginoliveoil 13.5 hotwater 58.21333333333333 fineseasalt 0.4,three,1.123491168022155761718750000000000000000000000000000000000000,0.887899756431579589843750000000000000000000000000000000000000,0.9985

In [147]:
df3.to_csv('Embeddings_files/subtraction_allrecipes_embeddings_with_counts.csv')

In [445]:
for w in df2['RECIPE'][0].split():
    
    print(w)

mayonnaise
parmigianoreggianocheese
granulatedsugar
romainelettuce
baconbits


In [71]:
def word_embed_p(wordstr):
    sum = 0
    
    if word in embedding_dict:
        for i,word in embedding_dict.items():
            sum+=i[word]
    return sum
            
    
    

In [119]:
words_embeddings['soup'].flatten()

array([-0.00240331,  0.03762205, -0.02749192, -0.04848162,  0.04228977,
       -0.01813643,  0.01563834, -0.03912672, -0.02333993,  0.0201702 ,
       -0.03688854, -0.02473341, -0.01886275, -0.03389432, -0.00428972,
        0.02012581,  0.04544756,  0.02659816, -0.00307486,  0.01334628,
        0.00992028, -0.03446176,  0.03543515,  0.00474365, -0.02860992,
       -0.04367108,  0.02054013,  0.00182164,  0.04817497,  0.02690083,
        0.01221266,  0.02760795, -0.01784338,  0.02309927, -0.03757583,
       -0.04131224, -0.00277395,  0.00288052, -0.01536344,  0.00434124,
        0.04562828, -0.04436678, -0.03301162, -0.01961676,  0.02871481,
       -0.04177602,  0.01416811, -0.03735606,  0.02146654, -0.01196771,
       -0.00378642, -0.00040765,  0.03887788,  0.02748281,  0.02077584,
        0.00117525,  0.03375361,  0.02505709, -0.04063374,  0.03086812,
       -0.03723745,  0.00302555,  0.02797593, -0.00828804,  0.03461227,
        0.00691745, -0.01064347,  0.03237707,  0.04385226,  0.00

In [478]:
original_model=Model(inputs=[desc_input,target_title_input,neg_title_input],outputs=[pos_title_logit,neg_title_logit])


original_model.compile(optimizer='adam',metrics=['accuracy'],
                       loss=['binary_crossentropy','binary_crossentropy'],loss_weights=[1.,1.])                    

In [479]:
original_model.fit_generator(trainGen,verbose=1,validation_data=validateGen,epochs=5, steps_per_epoch= 30,
                             workers=n_workers,max_queue_size=queue_size,use_multiprocessing=False,shuffle=True)


Epoch 1/5
30/30 [==============================] - 362s 12s/step - loss: 1.3836 - po_title_logit_loss: 0.6895 - ne_title_logit_loss: 0.6941 - po_title_logit_acc: 0.6250 - ne_title_logit_acc: 0.4503 - val_loss: 1.3797 - val_po_title_logit_loss: 0.6844 - val_ne_title_logit_loss: 0.6952 - val_po_title_logit_acc: 0.6983 - val_ne_title_logit_acc: 0.4325
Epoch 2/5
15/30 [==============>...............] - ETA: 53s - loss: 1.3794 - po_title_logit_loss: 0.6817 - ne_title_logit_loss: 0.6978 - po_title_logit_acc: 0.7408 - ne_title_logit_acc: 0.3737

KeyboardInterrupt: 

In [16]:
open('all_recipe_model.json'.format(model_name), 'w').write(original_model.to_json())


# In[ ]:


original_model.save_weights('all_recipe_model.hdf5'.format(name=model_name))